# Introduction

This notebook shows how to replace the `VolatileMemoryStore` memory storage used in a [previous notebook](./06-memory-and-embeddings.ipynb) with a `WeaviateMemoryStore`.

`WeaviateMemoryStore` is an example of a persistent (i.e. long-term) memory store backed by the Weaviate vector database.

# About Weaviate

[Weaviate](https://weaviate.io/) is an open-source vector database designed to scale seamlessly into billions of data objects. This implementation supports hybrid search out-of-the-box (meaning it will perform better for keyword searches).

You can run Weaviate in 5 ways:

- **SaaS** – with [Weaviate Cloud Services (WCS)](https://weaviate.io/pricing).

  WCS is a fully managed service that takes care of hosting, scaling, and updating your Weaviate instance. You can try it out for free with a sandbox that lasts for 14 days.

  To set up a SaaS Weaviate instance with WCS:

  1.  Navigate to [Weaviate Cloud Console](https://console.weaviate.cloud/).
  2.  Register or sign in to your WCS account.
  3.  Create a new cluster with the following settings:
      - `Subscription Tier` – Free sandbox for a free trial, or contact [hello@weaviate.io](mailto:hello@weaviate.io) for other options.
      - `Cluster name` – a unique name for your cluster. The name will become part of the URL used to access this instance.
      - `Enable Authentication?` – Enabled by default. This will generate a static API key that you can use to authenticate.
  4.  Wait for a few minutes until your cluster is ready. You will see a green tick ✔️ when it's done. Copy your cluster URL.

- **Hybrid SaaS**

  > If you need to keep your data on-premise for security or compliance reasons, Weaviate also offers a Hybrid SaaS option: Weaviate runs within your cloud instances, but the cluster is managed remotely by Weaviate. This gives you the benefits of a managed service without sending data to an external party.

  The Weaviate Hybrid SaaS is a custom solution. If you are interested in this option, please reach out to [hello@weaviate.io](mailto:hello@weaviate.io).

- **Self-hosted** – with a Docker container

  To set up a Weaviate instance with Docker:

  1. [Install Docker](https://docs.docker.com/engine/install/) on your local machine if it is not already installed.
  2. [Install the Docker Compose Plugin](https://docs.docker.com/compose/install/)
  3. Download a `docker-compose.yml` file with this `curl` command:

     ```
     curl -o docker-compose.yml "https://configuration.weaviate.io/v2/docker-compose/docker-compose.yml?modules=standalone&runtime=docker-compose&weaviate_version=v1.19.6"
     ```

     Alternatively, you can use Weaviate's docker compose [configuration tool](https://weaviate.io/developers/weaviate/installation/docker-compose) to generate your own `docker-compose.yml` file.

  4. Run `docker compose up -d` to spin up a Weaviate instance.

     > To shut it down, run `docker compose down`.

- **Self-hosted** – with a Kubernetes cluster

  To configure a self-hosted instance with Kubernetes, follow Weaviate's [documentation](https://weaviate.io/developers/weaviate/installation/kubernetes).|

# Setup

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.18.230725.3-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Weaviate, 0.18.230725.3-preview"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.Weaviate;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Orchestration;

First, we instantiate the Weaviate memory store. Uncomment ONE of the options below, depending on how you want to use Weaviate:
* from a Docker instance
* from WCS

In [ ]:
var apiKey = "my-secret-key";
WeaviateMemoryStore memoryStore = new("http://localhost:8080/v1/", apiKey);

Then, we register the memory store to the kernel:

In [ ]:
var openApiKey = "";
IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder
                       .WithOpenAITextCompletionService("text-davinci-003", openApiKey)
                       .WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", openApiKey)
                       .WithMemoryStorage(memoryStore)
                       .Build();
kernel.ImportSkill(new TextMemorySkill(kernel.Memory));

# Manually adding memories


Let's create some initial memories "About Me". We can add memories to our weaviate memory store by using `save_information_async`

In [ ]:
string collectionName = "AboutMe";

In [ ]:
async Task PopulateMemories(IKernel kernel)
{
    await kernel.Memory.SaveInformationAsync(collectionName, id: Guid.NewGuid().ToString(), text: "My name is Andrea");
    await kernel.Memory.SaveInformationAsync(collectionName, id: Guid.NewGuid().ToString(), text: "I currently work as a tour guide");
    await kernel.Memory.SaveInformationAsync(collectionName, id: Guid.NewGuid().ToString(), text: "I've been living in Seattle since 2005");
    await kernel.Memory.SaveInformationAsync(collectionName, id: Guid.NewGuid().ToString(), text: "I visited France and Italy five times since 2015");
    await kernel.Memory.SaveInformationAsync(collectionName, id: Guid.NewGuid().ToString(), text: "My family is from New York");
}

Searching is done through `SearchAsync`:

In [ ]:
async Task SearchMemoryExamples(IKernel kernel)
{
    string[] questions = new []
    {
        "what's my name",
        "where do I live?",
        "where's my family from?",
        "where have I traveled?",
        "what do I do for work"
    };
    
    foreach (var question in questions) {
        Console.WriteLine($"Question: {question}");
        var result = kernel.Memory.SearchAsync(collectionName, question, limit: 1);
        await foreach (var item in result)
        {
            Console.WriteLine("Answer:" + item.Metadata.Text);
        }
    }
}

Let's see the results of the functions:

In [ ]:
Console.WriteLine("Populating memory...");
await PopulateMemories(kernel);

Console.WriteLine("Asking questions... (manually)");
await SearchMemoryExamples(kernel);

Here's how to use the weaviate memory store in a chat application:

In [ ]:
async Task<Tuple<ISKFunction, SKContext>> SetupChatWithMemory(IKernel kernel)
{
    var prompt = @"""
            ChatBot can have a conversation with you about any topic.
            It can give explicit instructions or say 'I don't know' if
            it does not have an answer.

            Information about me, from previous conversations:
            - {{$fact1}} {{recall $fact1}}
            - {{$fact2}} {{recall $fact2}}
            - {{$fact3}} {{recall $fact3}}
            - {{$fact4}} {{recall $fact4}}
            - {{$fact5}} {{recall $fact5}}

            Chat:
            {{$chat_history}}
            User: {{$user_input}}
            ChatBot: """.Trim();

    var func = kernel.CreateSemanticFunction(prompt, maxTokens: 200, temperature: 0.8);
    var context = kernel.CreateNewContext();
    context["fact1"] = "what is my name?";
    context["fact2"] = "where do I live?";
    context["fact3"] = "where's my family from?";
    context["fact4"] = "where have I traveled?";
    context["fact5"] = "what do I do for work?";

    context[TextMemorySkill.CollectionParam] = collectionName;
    context[TextMemorySkill.RelevanceParam] = "0.8";

    context["chat_history"] = "";

    return Tuple.Create(func, context);
}

In [ ]:
using Microsoft.DotNet.Interactive;

async Task<bool> Chat(IKernel kernel, ISKFunction chatFunc, SKContext context)
{
    var userInput = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Chat (type exit to end):");
    Console.WriteLine("User:> " + userInput);
    context["user_input"] = userInput;

    if (userInput == "exit") {
        Console.WriteLine("Exiting chat...");
        return false;
    }

    var answer = await chatFunc.InvokeAsync(context);
    context["chat_history"] += $"\nUser:> {userInput}\nChatBot:> {answer}\n";
    Console.WriteLine("ChatBot:> " + answer);
    return true;
}

In [ ]:
Console.WriteLine("Setting up a chat (with memory!)");
var (chat_func, context) = await SetupChatWithMemory(kernel);

Console.WriteLine("Begin chatting (type 'exit' to exit):\n");
var chatting = true;
while (chatting)
     chatting = await Chat(kernel, chat_func, context);

# Adding documents to your memory

Create a dictionary to hold some files. The key is the hyperlink to the file and the value is the file's content:

In [ ]:
Dictionary<string, string> githubFiles = new();
githubFiles.Add("https://github.com/microsoft/semantic-kernel/blob/main/README.md", "README: Installation, getting started, and how to contribute");
githubFiles.Add("https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/02-running-prompts-from-file.ipynb", "Jupyter notebook describing how to pass prompts from a file to a semantic skill or function");
githubFiles.Add("https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/00-getting-started.ipynb", "Jupyter notebook describing how to get started with the Semantic Kernel");
githubFiles.Add("https://github.com/microsoft/semantic-kernel/tree/main/samples/skills/ChatSkill/ChatGPT", "Sample demonstrating how to create a chat skill interfacing with ChatGPT");
githubFiles.Add("https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/SemanticKernel/Memory/Volatile/VolatileMemoryStore.cs", "C# class that defines a volatile embedding store");
githubFiles.Add("https://github.com/microsoft/semantic-kernel/tree/main/samples/dotnet/KernelHttpServer/README.md", "README: How to set up a Semantic Kernel Service API using Azure Function Runtime v4");
githubFiles.Add("https://github.com/microsoft/semantic-kernel/tree/main/samples/apps/chat-summary-webapp-react/README.md", "README: README associated with a sample starter react-based chat summary webapp");

Use `SaveInformationAsync` to save the file:

In [ ]:
var collectionName = "SKGitHub";
Console.WriteLine("Adding some GitHub file URLs and their descriptions to a volatile Semantic Memory.");

foreach (var key in githubFiles.Keys) {
    await kernel.Memory.SaveInformationAsync(collectionName, key, Guid.NewGuid().ToString(), description: githubFiles[key]);
}

Use `SearchAsync` to ask a question:

In [ ]:
var ask = "I love Jupyter notebooks, how should I get started?";
Console.WriteLine("===========================\n" + "Query: " + ask + "\n");

var memories = kernel.Memory.SearchAsync(collectionName, ask, limit: 7, minRelevanceScore: 0.77);
 
await foreach (var item in memories)
{
    Console.WriteLine(item.Metadata.Text + " " + item.Metadata.Description);
}